In [0]:
%pip install osmnx==1.7.1 networkx==3.2.1 geopy==2.4.1 shapely==2.0.3 databricks-sdk[openai]>=0.35.0

In [0]:
import json

try:
    sim_cfg_json = dbutils.widgets.get("sim_cfg_json")
except:
    print("no widget")
    sim_cfg_json = ''
    
SIM_CFG = json.loads(sim_cfg_json) if sim_cfg_json != '' else json.load(open('./config.json'))

In [0]:
"""
Ghost-kitchen menu generator
————————————
• Creates four relational datasets: brands, menus, categories, items
• Adds deterministic, “reasonable” USD prices to each item
"""

import pandas as pd
import hashlib
from pathlib import Path

# ---------- 1. SOURCE DATA (renamed real-world menus) ----------
brands_data = [
    ("McDoodles", {
        "Burgers": ["Big Stack", "Quarter Poundish", "Double Quarter Poundish", "Cheese Burgerette"],
        "Chicken & Fish Sandwiches": ["McCrisper", "Spicy McCrisper", "Filet-O-Fishish", "McChicklet"],
        "Nuggets": ["Chick Nuggies 6pc"],
        "Fries & Sides": ["Famous Frites", "Apple Slice Bites"],
        "Breakfast": ["Egg McMuffle", "Bacon Biscuit"],
        "Coffees & Beverages": ["Iced Brew", "Fizzy Cola"]
    }),
    ("ChipoLot", {
        "Burritos": ["Chicken Burrito", "Steak Burrito", "Barbacoa Burrito", "Veggie Burrito"],
        "Bowls": ["Chicken Bowl", "Keto Bowl", "High Protein Bowl"],
        "Tacos": ["Chicken Tacos", "Carnitas Tacos"],
        "Salads": ["Supergreens Salad"],
        "Kids": ["Build-Your-Own Kid Meal"],
        "Sides & Extras": ["Chips & Guac", "Chips & Queso"]
    }),
    ("Shack Stack", {
        "Burgers": ["ShackBurger Single", "ShackBurger Double", "Smoke Stack Single", "Veggie Stack"],
        "Chicken": ["Chicken Shack", "Avocado Bacon Chicken"],
        "Dogs": ["Flat-Top Dog"],
        "Fries": ["Crinkle Fries", "Cheese Fries"],
        "Shakes": ["Vanilla Shake", "Chocolate Shake"],
        "Drinks": ["Shack-made Lemonade"]
    }),
    ("GreenSprout", {
        "Bowls": ["Harvest Bowl", "Crispy Rice Bowl", "Shroomami"],
        "Salads": ["Kale Caesar", "Guac Greens", "Buffalo Chicken Salad"],
        "Protein Plates": ["Garlic Steak Plate", "Hot Honey Chicken Plate"],
        "Kids": ["Little Harvest"],
        "Sides": ["Rosemary Focaccia", "Hummus & Focaccia"],
        "Drinks": ["Jasmine Green Tea"]
    }),
    ("Pando Dash", {
        "Entrees": ["Orange Chickadee", "Kung Pow Chicken", "Beijing Beefy", "Honey Walnut Shrimpish"],
        "Sides": ["Chow Mein Noodles", "Fried Rice"],
        "Appetizers": ["Cream Cheese Rangoon", "Chicken Egg Roll"],
        "Drinks": ["Fountain Soda"],
        "Bundles": ["Bowl Combo"]
    }),
    ("Kava Kitchen", {
        "Bowls": ["Spicy Lamb & Avo Bowl", "Falafel Crunch Bowl", "Chicken + Rice Bowl"],
        "Pitas": ["Steak & Feta Pita", "Crispy Falafel Pita"],
        "Kids": ["Mini Pita Meal"],
        "Drinks": ["Strawberry Citrus Lemonade"],
        "Sides": ["Hot Harissa Chips"],
        "Desserts": ["Salted Choco Cookie"]
    }),
    ("MaruNoodle", {
        "Udon": ["Beef Udon", "Chicken Katsu Udon", "Curry Udon"],
        "Rice Bowls": ["Gyudon Rice", "Yakitori Rice"],
        "Tempura": ["Shrimp Tempura", "Sweet Potato Tempura"],
        "Onigiri": ["Spam Musubi Onigiri"],
        "Add-Ons": ["BK Sauce"]
    }),
    ("Jinya Noodle Bar", {
        "Ramen": ["Tonkotsu Ramen", "Spicy Chicken Ramen", "Vegan Ramen"],
        "Small Plates": ["Pork Gyoza", "Takoyaki"],
        "Rice Bowls": ["Chicken Karaage Rice Bowl"],
        "Toppings": ["Extra Chashu", "Spicy Sauce"],
        "Drinks": ["Green Tea"]
    }),
    ("Fo Ho", {
        "Pho": ["Pho Combo", "Pho Chicken"],
        "Rolls": ["Fried Spring Roll", "Summer Roll"],
        "Rice Plates": ["Grilled Pork Rice Plate"],
        "Vermicelli": ["Lemongrass Chicken Vermicelli"],
        "Drinks": ["Vietnamese Iced Coffee"]
    }),
    ("Yo! Sushii", {
        "Maki": ["California Roll", "Spicy Tuna Roll"],
        "Nigiri": ["Salmon Nigiri", "Ebi Nigiri"],
        "Sashimi": ["Tuna Sashimi"],
        "Street Food": ["Chicken Katsu Curry", "Yakisoba", "Miso Soup"],
        "Desserts": ["Mochi Ice Cream"],
        "Drinks": ["Matcha Soda"]
    }),
    ("Taco Ring", {
        "Tacos": ["Crunchy Taco", "Soft Taco Supreme"],
        "Burritos": ["Beefy 5-Layer Burrito"],
        "Specialties": ["Doritos Locos Taco"],
        "Quesadillas": ["Chicken Quesadilla"],
        "Nachos": ["Nacho Bellgrande"],
        "Sides": ["Cinnamon Twists"],
        "Drinks": ["Baja Blast Soda"]
    }),
    ("Starbrews", {
        "Hot Coffee": ["Caffè Latte", "Pike Place Brew"],
        "Cold Coffee": ["Cold Brew"],
        "Refreshers": ["Strawberry Açaí Refresher"],
        "Frappuccinos": ["Caramel Frappuccino"],
        "Bakery": ["Plain Bagel"],
        "Breakfast Sandwiches": ["Sausage Cheddar Egg Sandwich"],
        "Tea": ["Green Tea"]
    }),
    ("Dominni Pizza", {
        "Specialty Pizzas": ["ExtravaganZZza Pizza", "MeatZZa Pizza", "Pacific Veggie Pizza"],
        "Build Your Own Pizza": ["Custom Pizza"],
        "Pasta": ["Chicken Alfredo Pasta"],
        "Chicken": ["Boneless Chicken"],
        "Sides": ["Bread Twists"],
        "Desserts": ["Chocolate Lava Cake"],
        "Drinks": ["Coke"]
    }),
    ("Five Gals", {
        "Burgers": ["Bacon Cheeseburger", "Little Cheeseburger"],
        "Dogs": ["Kosher Style Hot Dog"],
        "Sandwiches": ["BLT Sandwich"],
        "Fries": ["Cajun Fries"],
        "Milkshakes": ["Vanilla Milkshake"],
        "Drinks": ["Bottled Soda"]
    }),
    ("PokeCraft", {
        "Poke Bowls": ["Signature Hawaiian Bowl", "Spicy Ahi Bowl"],
        "Burritos": ["Chicken Katsu Burrito"],
        "Salads": ["Vegan Salad"],
        "Sides": ["Miso Soup Side"],
        "Drinks": ["Japanese Soda"]
    })
]

# ---------- 2. PRICE BANDS BY CATEGORY KEYWORD ----------
pricing_rules = [
    (["pizza"], (12, 18)),
    (["burger"], (6, 11)),
    (["burrito"], (7, 10)),
    (["bowl"], (9, 12)),
    (["udon", "ramen"], (10, 14)),
    (["salad"], (8, 11)),
    (["pho", "rice plate", "plate"], (9, 13)),
    (["kids"], (4, 6)),
    (["fries"], (2, 4)),
    (["side", "sides"], (2, 5)),
    (["drink", "coffee", "tea", "soda", "lemonade"], (2, 4)),
    (["shake", "frappuccino"], (4, 6)),
    (["dessert", "mochi", "cookie", "cake"], (3, 5)),
    (["sandwich"], (4, 7)),
    (["dog"], (3, 6)),
    (["nugget"], (3, 5)),
    (["taco"], (2, 4)),
    (["quesa", "nacho"], (3, 6)),
    (["roll", "nigiri"], (4, 7)),
    (["sashimi"], (5, 9)),
    (["tempura"], (4, 7)),
    (["onigiri"], (3, 5)),
    (["rice"], (8, 11)),
]

def deterministic_random(item_name: str, low: int, high: int) -> float:
    """Hash item name to a deterministic price within [low, high)."""
    h = hashlib.md5(item_name.encode()).hexdigest()
    cents_range = (high - low) * 100
    return round(low + (int(h[:8], 16) % cents_range) / 100, 2)

def get_price(item_name: str, category_name: str) -> float:
    lower_cat = category_name.lower()
    for keywords, (lo, hi) in pricing_rules:
        if any(k in lower_cat for k in keywords):
            return deterministic_random(item_name, lo, hi)
    return deterministic_random(item_name, 6, 10)  # fallback

# ---------- 3. BUILD TABLE ROWS ----------
brands_rows, menus_rows, categories_rows, items_rows = [], [], [], []

brand_id = menu_id = category_id = item_id = 1
for brand, menu_struct in brands_data:
    brands_rows.append({"id": brand_id, "name": brand})
    menus_rows.append({"id": menu_id, "brand_id": brand_id, "name": f"{brand} Main Menu"})
    for cat, items in menu_struct.items():
        categories_rows.append({"id": category_id, "menu_id": menu_id, "brand_id": brand_id,"name": cat})
        for it in items:
            price = get_price(it, cat)
            items_rows.append({
                "id": item_id,
                "category_id": category_id,
                "menu_id": menu_id,
                "brand_id": brand_id,
                "name": it,
                "price": price
            })
            item_id += 1
        category_id += 1
    brand_id += 1
    menu_id += 1

# ---------- 4. TO DATAFRAMES ----------
brands_df     = pd.DataFrame(brands_rows)
menus_df      = pd.DataFrame(menus_rows)
categories_df = pd.DataFrame(categories_rows)
items_df      = pd.DataFrame(items_rows)


In [0]:
catalog = SIM_CFG['catalog']
schema = SIM_CFG['schema']
table = SIM_CFG['table']

spark.sql(f"DROP SCHEMA IF EXISTS {catalog}.{schema} CASCADE")
spark.sql(f"CREATE SCHEMA {catalog}.{schema}")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog}.{schema}.brand (
  id   BIGINT NOT NULL,
  name STRING NOT NULL,
  CONSTRAINT brand_pk PRIMARY KEY (id)
)""")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog}.{schema}.menu (
  id       BIGINT NOT NULL,
  name     STRING NOT NULL,
  brand_id BIGINT NOT NULL,

  CONSTRAINT menu_pk        PRIMARY KEY (id),
  CONSTRAINT menu_brand_fk  FOREIGN KEY (brand_id)
    REFERENCES {catalog}.{schema}.brand(id)
)""")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog}.{schema}.category (
  id       BIGINT NOT NULL,
  name     STRING NOT NULL,
  menu_id  BIGINT NOT NULL,
  brand_id BIGINT NOT NULL,

  CONSTRAINT category_pk         PRIMARY KEY (id),
  CONSTRAINT category_menu_fk    FOREIGN KEY (menu_id)
      REFERENCES {catalog}.{schema}.menu(id),
  CONSTRAINT category_brand_fk   FOREIGN KEY (brand_id)
      REFERENCES {catalog}.{schema}.brand(id)
)""")

spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog}.{schema}.item (
  id          BIGINT NOT NULL,
  name        STRING NOT NULL,
  description STRING,
  price       DOUBLE NOT NULL,
  image_data  STRING,

  brand_id    BIGINT NOT NULL,
  menu_id     BIGINT NOT NULL,
  category_id BIGINT NOT NULL,

  CONSTRAINT item_pk            PRIMARY KEY (id),
  CONSTRAINT item_brand_fk      FOREIGN KEY (brand_id)
      REFERENCES {catalog}.{schema}.brand(id),
  CONSTRAINT item_menu_fk       FOREIGN KEY (menu_id)
      REFERENCES {catalog}.{schema}.menu(id),
  CONSTRAINT item_category_fk   FOREIGN KEY (category_id)
      REFERENCES {catalog}.{schema}.category(id)
)""")

spark.sql(f"""
CREATE TABLE {catalog}.{schema}.{table} (
  event_id   STRING,
  event_type STRING,
  ts         TIMESTAMP,
  gk_id      STRING,
  order_id   STRING,
  sequence   INT,
  body       STRING,
  day        DATE
)
USING delta
PARTITIONED BY (day)
TBLPROPERTIES(delta.minReaderVersion=2, delta.minWriterVersion=5)""")

spark.sql(f"""CREATE VOLUME {catalog}.{schema}.cache""")


In [0]:
spark.createDataFrame(brands_df).write.mode("overwrite").saveAsTable("brand")
spark.createDataFrame(menus_df).write.mode("overwrite").saveAsTable("menu")
spark.createDataFrame(categories_df).write.mode("overwrite").saveAsTable("category")
spark.createDataFrame(items_df).write.mode("overwrite").saveAsTable("item")

In [0]:
# ╔═══════════════════════════════════════════════════════════════════════╗
# ║  Ghost-Kitchen Event Simulator 2.1  →  Delta table                    ║
# ╚═══════════════════════════════════════════════════════════════════════╝
#
#  All tunables live in SIM_CFG  – supply as a JSON widget or edit below
# ------------------------------------------------------------------------

import asyncio, datetime as dt, json, math, pickle, random, time, uuid
from pathlib import Path
from typing import Dict, List, Tuple

import nest_asyncio, networkx as nx, numpy as np, osmnx as ox, pandas as pd
from pyspark.sql import SparkSession, functions as F

nest_asyncio.apply()
spark = SparkSession.builder.getOrCreate()

# ─────────────────────────────────────────────────────────────────────────
# 0.  CONFIGURATION                                                      │
# ─────────────────────────────────────────────────────────────────────────
CFG = SIM_CFG; get = lambda k: CFG[k]
RAND = random.Random(get("random_seed")); np.random.seed(get("random_seed"))

CATALOG, SCHEMA, TABLE = get("catalog"), get("schema"), get("table")
TABLE_FQN              = f"{CATALOG}.{SCHEMA}.{TABLE}"
START_TS, END_TS       = map(lambda t: dt.datetime.strptime(t,"%Y-%m-%d %H:%M:%S"),
                             (get("start_ts"), get("end_ts")))
SPEED_UP               = get("speed_up")
GK_ADDRESS, GK_R_MI    = get("gk_location"), get("radius_mi")
GK_DRIVER_MPH          = get("driver_mph")
NOISE, SVC             = get("noise_pct")/100, get("svc")
BATCH_ROWS, BATCH_SEC  = get("batch_rows"), get("batch_seconds")
PING_SEC               = get("ping_sec")

# ─────────────────────────────────────────────────────────────────────────
# 1.  ROAD GRAPH + NODES (cached)                                        │
# ─────────────────────────────────────────────────────────────────────────
CACHE_DIR = f"/Volumes/{CATALOG}/{SCHEMA}/cache"
GRAPH_PKL, NODES_PARQ = Path(CACHE_DIR, "roadGraph.pkl"), Path(CACHE_DIR, "nodes.parquet")
Path(CACHE_DIR).mkdir(parents=True, exist_ok=True)

def load_graph() -> nx.MultiDiGraph:
    if GRAPH_PKL.exists(): return pickle.loads(GRAPH_PKL.read_bytes())
    ox.settings.log_console = False
    g = ox.graph_from_point(ox.geocoder.geocode(GK_ADDRESS),
                            dist=GK_R_MI*1609.34, network_type="drive")
    GRAPH_PKL.write_bytes(pickle.dumps(g)); return g

def load_nodes(g) -> pd.DataFrame:
    if NODES_PARQ.exists(): return pd.read_parquet(NODES_PARQ)
    rows=[]
    for nid,data in g.nodes(data=True):
        h,s = data.get("addr:housenumber"), data.get("addr:street")
        label = f"{h} {s}" if h and s else f"{RAND.randint(1,9999)} Main St"
        rows.append(dict(node_id=nid, lat=data["y"], lon=data["x"], addr=label))
    df = pd.DataFrame(rows); df.to_parquet(NODES_PARQ, index=False); return df

GRAPH = load_graph(); NODES = load_nodes(GRAPH)
GK_LAT, GK_LON = ox.geocoder.geocode(GK_ADDRESS)
GK_NODE = ox.distance.nearest_nodes(GRAPH, GK_LON, GK_LAT)
comp_map = {n:cid for cid,comp in enumerate(nx.connected_components(GRAPH.to_undirected())) for n in comp}
NODES = NODES[NODES.node_id.map(comp_map)==comp_map[GK_NODE]].reset_index(drop=True)
def rand_customer():
    r = NODES.sample(1, random_state=RAND.randrange(2**32)).iloc[0]  # ✔
    return int(r.node_id), r.lat, r.lon, r.addr

# ─────────────────────────────────────────────────────────────────────────
# 2.  MENU + BRAND MOMENTUM                                              │
# ─────────────────────────────────────────────────────────────────────────
ITEMS_DF = spark.table("item").toPandas()
ITEMS_BY_BRAND = {bid: grp.to_dict("records") for bid, grp in ITEMS_DF.groupby("brand_id")}
BRANDS = list(ITEMS_BY_BRAND); RAND.shuffle(BRANDS)

bm = get("brand_momentum"); cuts = np.cumsum([bm["improving"], bm["flat"]])*len(BRANDS)
IMPR, FLAT, DECL = BRANDS[:int(cuts[0])], BRANDS[int(cuts[0]):int(cuts[1])], BRANDS[int(cuts[1]):]
rates = get("momentum_rates")

def brand_weight(day, total, b):
    if b in IMPR:  f = (1+rates["growth"])**(day/30)
    elif b in DECL: f = (1-rates["decline"])**(day/30)
    else:          f = 1.0
    return f

def rand_basket(day, total, p_single=0.7, max_brands=4, items_rng=(1,4)):
    w = np.array([brand_weight(day,total,b) for b in BRANDS]); w = w/w.sum()
    chosen = [RAND.choices(BRANDS, weights=w, k=1)[0]] if RAND.random()<p_single \
             else RAND.choices(BRANDS, weights=w, k=RAND.randint(2,max_brands))
    items=[]
    for b in chosen:
        for itm in RAND.sample(ITEMS_BY_BRAND[b], RAND.randint(*items_rng)):
            rec = itm.copy(); rec["qty"] = RAND.randint(1,3); items.append(rec)
    return items

# ─────────────────────────────────────────────────────────────────────────
# 3.  DEMAND SHAPE                                                       │
# ─────────────────────────────────────────────────────────────────────────
def minute_weights():
    w = np.ones(1440)
    for s,e,m in [("11:00","13:30",3), ("17:00","20:00",3.5)]:
        s_dt,e_dt = [dt.datetime.strptime(x,"%H:%M") for x in (s,e)]
        s_m,e_m = s_dt.hour*60+s_dt.minute, e_dt.hour*60+e_dt.minute; span=e_m-s_m
        for mi in range(s_m,e_m):
            x = (mi-s_m)/span; w[mi] += (m-1)*(math.sin(math.pi*x)**2)
    return w
MIN_W = minute_weights()

def orders_today(d, total, date):
    base = CFG["orders_day_1"]+(CFG["orders_last"]-CFG["orders_day_1"])*(d/total)
    wd = {"mon":1,"tue":1.05,"wed":1.08,"thu":1.10,"fri":1.25,"sat":1.35,"sun":1.15}[date.strftime("%a").lower()]
    return base*wd*RAND.uniform(1-NOISE,1+NOISE)

# ─────────────────────────────────────────────────────────────────────────
# 4.  ROUTING                                                            │
# ─────────────────────────────────────────────────────────────────────────
def shortest_route(lat,lon):
    cust = ox.distance.nearest_nodes(GRAPH, lon, lat)
    try:
        path = nx.shortest_path(GRAPH, GK_NODE, cust, weight="length"); g = GRAPH
    except nx.NetworkXNoPath:
        g = GRAPH.to_undirected(); path = nx.shortest_path(g, GK_NODE, cust, weight="length")
    coords = [(g.nodes[n]["y"], g.nodes[n]["x"]) for n in path]
    dist = sum(min(d["length"] for d in g[u][v].values()) for u,v in zip(path[:-1],path[1:]))
    return coords, dist

# ─────────────────────────────────────────────────────────────────────────
# 5.  DELTA WRITER + DATA-QUALITY                                        │
# ─────────────────────────────────────────────────────────────────────────
EVENT_Q = asyncio.PriorityQueue(); CNT = 0
GK_ID = uuid.uuid4().hex; DQ = get("dq")

def maybe_corrupt(ev, payload):
    dq = DQ.get(ev, {}); return {k:(None if RAND.random()<dq.get(k,0) else v) for k,v in payload.items()}

def enqueue(ts, ev, oid, seq, payload):
    global CNT; CNT += 1
    EVENT_Q.put_nowait((ts.timestamp(), CNT, {
        "event_id": uuid.uuid4().hex,
        "event_type": ev,
        "ts": ts.strftime("%Y-%m-%d %H:%M:%S.%f"),
        "gk_id": GK_ID,
        "order_id": oid,
        "sequence": seq,
        "body": json.dumps(maybe_corrupt(ev, payload))
    }))

def flush(rows):
    if not rows: return
    (spark.createDataFrame(rows)
          .withColumn("ts", F.to_timestamp("ts","yyyy-MM-dd HH:mm:ss.SSSSSS"))
          .withColumn("day",F.to_date("ts"))
          .withColumn("sequence",F.col("sequence").cast("INT"))
          .write.format("delta").mode("append").saveAsTable(TABLE_FQN))

async def consumer():
    buf,last=[],time.time()
    while True:
        _,_,row = await EVENT_Q.get(); buf.append(row)
        if len(buf)>=BATCH_ROWS or (time.time()-last)>=BATCH_SEC:
            flush(buf); buf.clear(); last=time.time()

# ─────────────────────────────────────────────────────────────────────────
# 6.  ORDER LIFECYCLE (real-time)                                        │
# ─────────────────────────────────────────────────────────────────────────
MICRO = lambda n: dt.timedelta(microseconds=n)
def gauss(mu_sigma): return max(0.1, RAND.gauss(*mu_sigma))

driver_cfg = get("driver_arrival")
def driver_arrival_time(created_at, t_ready, t_pick):
    """Return a dt between created_at and t_pick with prob mass after ready."""
    if RAND.random() < driver_cfg["after_ready_pct"]:
        base, span = t_ready, (t_pick - t_ready)
    else:
        base, span = created_at, (t_ready - created_at)
    frac = np.random.beta(driver_cfg["alpha"], driver_cfg["beta"])
    t_arr = base + span * frac
    if t_arr >= t_pick:  # safety: keep strict order
        t_arr = t_pick - MICRO(1)
    return t_arr

async def play_order(created_at, day, total):
    oid=uuid.uuid4().hex; seq=0
    _,lat,lon,addr = rand_customer(); items = rand_basket(day,total)
    pts,dist = shortest_route(lat,lon); drive_min = dist/1609.34/GK_DRIVER_MPH*60

    t_cs  = created_at + dt.timedelta(minutes=gauss(SVC["cs"]))
    t_sf  = t_cs + dt.timedelta(minutes=gauss(SVC["sf"]))
    t_fr  = t_sf + dt.timedelta(minutes=gauss(SVC["fr"]))
    t_ready = t_fr
    t_pick  = t_ready + dt.timedelta(minutes=gauss(SVC["rp"]))
    t_drop  = t_pick  + dt.timedelta(minutes=drive_min)
    t_arr   = driver_arrival_time(created_at, t_ready, t_pick)

    enqueue(created_at+MICRO(seq),"order_created",oid,seq,
            dict(customer_lat=lat,customer_lon=lon,customer_addr=addr,items=items)); seq+=1
    enqueue(t_cs+MICRO(seq),"gk_started",oid,seq,{}); seq+=1
    enqueue(t_sf+MICRO(seq),"gk_finished",oid,seq,{}); seq+=1
    enqueue(t_ready+MICRO(seq),"gk_ready",oid,seq,{}); seq+=1
    enqueue(t_arr+MICRO(seq),"driver_arrived",oid,seq,{}); seq+=1
    enqueue(t_pick+MICRO(seq),"driver_picked_up",oid,seq,
            dict(route_points=pts, eta_mins=round(drive_min,1))); seq+=1

    hops=max(1,int(drive_min*60//PING_SEC))
    for h in range(1,hops):
        p=h/hops; lat_i,lon_i=pts[int(p*(len(pts)-1))]
        enqueue(t_pick+dt.timedelta(seconds=h*PING_SEC)+MICRO(seq),"driver_ping",oid,seq,
                dict(progress_pct=round(p*100,1),loc_lat=lat_i,loc_lon=lon_i)); seq+=1

    enqueue(t_drop+MICRO(seq),"delivered",oid,seq,
            dict(delivered_lat=lat,delivered_lon=lon))

# ─────────────────────────────────────────────────────────────────────────
# 7.  BACK-FILL (single write)                                           │
# ─────────────────────────────────────────────────────────────────────────
def build_rows(ts, lat, lon, addr, items, pts, dist, svc, day, total):
    rows=[]; seq=0; oid=uuid.uuid4().hex
    drive=dist/1609.34/GK_DRIVER_MPH*60
    t_cs=ts+dt.timedelta(minutes=gauss(svc["cs"]))
    t_sf=t_cs+dt.timedelta(minutes=gauss(svc["sf"]))
    t_fr=t_sf+dt.timedelta(minutes=gauss(svc["fr"]))
    t_ready=t_fr
    t_pick=t_ready+dt.timedelta(minutes=gauss(svc["rp"]))
    t_drop=t_pick+dt.timedelta(minutes=drive)
    t_arr=driver_arrival_time(ts, t_ready, t_pick)

    def add(t,ev,p):
        nonlocal seq
        rows.append({
            "event_id":uuid.uuid4().hex,"event_type":ev,
            "ts":t.strftime("%Y-%m-%d %H:%M:%S.%f"),"gk_id":GK_ID,
            "order_id":oid,"sequence":seq,
            "body":json.dumps(maybe_corrupt(ev,p))
        }); seq+=1

    add(ts,"order_created",dict(customer_lat=lat,customer_lon=lon,customer_addr=addr,items=items))
    add(t_cs,"gk_started",{}); add(t_sf,"gk_finished",{}); add(t_ready,"gk_ready",{})
    add(t_arr,"driver_arrived",{}); add(t_pick,"driver_picked_up",dict(route_points=pts,eta_mins=round(drive,1)))
    hops=max(1,int(drive*60//PING_SEC))
    for h in range(1,hops):
        p=h/hops; lat_i,lon_i=pts[int(p*(len(pts)-1))]
        add(t_pick+dt.timedelta(seconds=h*PING_SEC),"driver_ping",
            dict(progress_pct=round(p*100,1),loc_lat=lat_i,loc_lon=lon_i))
    add(t_drop,"delivered",dict(delivered_lat=lat,delivered_lon=lon))
    return rows

def gen_backfill(start, now, total_days):
    back=[]
    for d in range(total_days+1):
        date = start.date()+dt.timedelta(days=d)
        mean = orders_today(d,total_days,date)
        lam  = mean/MIN_W.sum()*MIN_W; midnight=dt.datetime.combine(date,dt.time.min)
        for m,v in enumerate(lam):
            for _ in range(np.random.poisson(v)):
                ts=midnight+dt.timedelta(minutes=m,seconds=RAND.randint(0,59))
                if ts>=now: continue
                _,lat,lon,addr = rand_customer(); items=rand_basket(d,total_days)
                pts,dist=shortest_route(lat,lon)
                back.extend(build_rows(ts,lat,lon,addr,items,pts,dist,SVC,d,total_days))
    return back

# ─────────────────────────────────────────────────────────────────────────
# 8.  MAIN SCHEDULER                                                     │
# ─────────────────────────────────────────────────────────────────────────
async def schedule():
    now = dt.datetime.utcnow()
    total_days = (END_TS.date()-START_TS.date()).days

    flush(gen_backfill(START_TS, now, total_days))   # back-fill
    asyncio.create_task(consumer())                  # live consumer

    async def later(ts, d):
        await asyncio.sleep(max(0,(ts-now).total_seconds()/SPEED_UP))
        await play_order(ts, d, total_days)

    futures=[]
    for d in range(total_days+1):
        date = START_TS.date()+dt.timedelta(days=d)
        mean = orders_today(d,total_days,date)
        lam  = mean/MIN_W.sum()*MIN_W; midnight = dt.datetime.combine(date,dt.time.min)
        for m,v in enumerate(lam):
            for _ in range(np.random.poisson(v)):
                ts=midnight+dt.timedelta(minutes=m,seconds=RAND.randint(0,59))
                if ts>=now:
                    futures.append(asyncio.create_task(later(ts,d)))
    if futures: await asyncio.gather(*futures)

print(f"👻  GK-sim → {TABLE_FQN}  (×{SPEED_UP})")
print(f"Running with config: {CFG}")
await schedule()
